In [ ]:
import eval_my_mlips.utils as emu
from mace.calculators import MACECalculator

In [ ]:
# So if I'm using mps, neead Float32
calculator = MACECalculator(model_paths="../models/2023-12-10-mace-128-L0_energy_epoch-249.model", device="mps") # small model
# TypeError: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.

In [ ]:
# still doesn't work
calculator = MACECalculator(model_paths="../models/2023-12-10-mace-128-L0_energy_epoch-249.model",
                            device="mps",
                            default_dtype = "float32") # small model

In [ ]:
calculator = MACECalculator(model_paths="../models/2023-12-10-mace-128-L0_energy_epoch-249.model")

In [ ]:
data_dir = "/Users/swyant/cesmix/datasets/HfO2_cesmix/april_2025_data_mv/processed_and_split/"

In [ ]:
reference_configs = emu.load_configurations(data_dir)

So I key oversight, I thought you'd be able to pass in E0s, to at least get a sense of the raw initial error for the model, but I think they are just baked into the model. in the same way that the model. This will make this initial pass kind of useless

In [ ]:
import torch

In [ ]:
small_model = torch.load(f="../models/2023-12-10-mace-128-L0_energy_epoch-249.model", map_location="cpu")

In [ ]:
print(small_model)

In [ ]:
small_model.float()

In [ ]:
torch.save(small_model, "../models/small_float32.model")

In [ ]:
calc_f32 = calculator = MACECalculator(model_paths="../models/small_float32.model",
                            device="mps",
                            default_dtype = "float32")

I feel like I shouldn't need to do that, but I think saving it to float32 worked?

In [ ]:
small_model.__dict__

In [ ]:
predicted_configs = emu.compute_predictions(reference_configs, calc_f32)

Nope, still doesn't work

In [ ]:
predicted_configs = emu.compute_predictions(reference_configs, calculator)

In [ ]:
small_model.__dict__

In [ ]:
sample_config = reference_configs["HfOx_test_0K"][0].copy()

In [ ]:
sample_config.calc = calculator

In [ ]:
model_atomic_numbers = calculator.z_table.zs

In [ ]:
# Get the tensor of isolated energies [num_elements, num_heads]
atomic_energies_tensor = calculator.models[0].atomic_energies_fn.atomic_energies.cpu().numpy()


In [ ]:
atomic_energies_tensor

In [ ]:
calculator.available_heads.index(calculator.head)

In [ ]:
energy_map = dict(zip(model_atomic_numbers, atomic_energies_tensor[:]))

In [ ]:
energy_map

In [ ]:
config_atomic_numbers = sample_config.get_atomic_numbers()

In [ ]:
config_atomic_numbers

In [ ]:
total_isolated_energy = sum([energy_map[z] for z in config_atomic_numbers])

In [ ]:
binding_energy = total_energy - total_isolated_energy

In [ ]:
total_energy = sample_config.get_potential_energy()


In [ ]:
sample_config.calc.results["node_energy"].sum()

In [ ]:
binding_energy = total_energy - total_isolated_energy

In [ ]:
binding_energy

In [ ]:
new_sample_config = sample_config.copy()

In [ ]:
new_sample_config.calc = calc_f32

In [ ]:
new_sample_config.get_potential_energy()